## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,40.28,93,20,4.61,few clouds
1,1,Hermanus,ZA,-34.4187,19.2345,63.81,78,87,14.18,overcast clouds
2,2,Vila Franca Do Campo,PT,37.7167,-25.4333,73.81,67,13,3.09,few clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,50.11,46,20,40.29,few clouds
4,4,Veracruz,MX,19.3333,-96.6667,68.90,82,99,2.26,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vila Franca Do Campo,PT,37.7167,-25.4333,73.81,67,13,3.09,few clouds
9,9,Guerrero Negro,MX,27.9769,-114.0611,75.87,60,55,14.27,broken clouds
10,10,Saint Anthony,US,45.0205,-93.2180,70.14,85,90,4.61,overcast clouds
13,13,Cervo,ES,43.6702,-7.4101,73.20,76,1,12.91,clear sky
20,20,Rikitea,PF,-23.1203,-134.9692,73.76,75,76,7.63,broken clouds
21,21,Vaini,TO,-21.2000,-175.2000,75.45,81,1,8.10,clear sky
24,24,Along,IN,28.1667,94.7667,71.44,97,13,1.66,few clouds
25,25,Freeport,US,40.6576,-73.5832,78.96,66,96,1.99,overcast clouds
31,31,Tikaitnagar,IN,26.9500,81.5833,77.74,64,0,1.86,clear sky
32,32,Sao Joao Da Barra,BR,-21.6403,-41.0511,73.49,70,100,4.81,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                165
City                   165
Country                165
Lat                    165
Lng                    165
Max Temp               165
Humidity               165
Cloudiness             165
Wind Speed             165
Current Description    165
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vila Franca Do Campo,PT,73.81,few clouds,37.7167,-25.4333,
9,Guerrero Negro,MX,75.87,broken clouds,27.9769,-114.0611,
10,Saint Anthony,US,70.14,overcast clouds,45.0205,-93.2180,
13,Cervo,ES,73.20,clear sky,43.6702,-7.4101,
20,Rikitea,PF,73.76,broken clouds,-23.1203,-134.9692,
21,Vaini,TO,75.45,clear sky,-21.2000,-175.2000,
24,Along,IN,71.44,few clouds,28.1667,94.7667,
25,Freeport,US,78.96,overcast clouds,40.6576,-73.5832,
31,Tikaitnagar,IN,77.74,clear sky,26.9500,81.5833,
32,Sao Joao Da Barra,BR,73.49,overcast clouds,-21.6403,-41.0511,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame
 # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
  
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vila Franca Do Campo,PT,73.81,few clouds,37.7167,-25.4333,Pestana Bahia Praia
9,Guerrero Negro,MX,75.87,broken clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
10,Saint Anthony,US,70.14,overcast clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
13,Cervo,ES,73.20,clear sky,43.6702,-7.4101,Hotel Spa Galatea
20,Rikitea,PF,73.76,broken clouds,-23.1203,-134.9692,People ThankYou


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))